# Face to Cartoon with cycleGAN

## Group Project in Advanced Topics of Machine Learning (FS2019)

##### Guodong Zeng, Benjamin Fankhauser, Jan Segessenmann, Gautam Ilango

### Introduction 

Our goal is to switch image styles from a real faces to cartoon faces using unpaired data only. Some of the important features of the real faces should be preserved and recognized in the generated cartoon faces.  
As a starting point, we use an implementation of cycleGAN introduced by Jun-Yan Zhu et al. [1] cycleGAN achieves impressive results in learning the mapping between different styles of images (e.g. paintings $\to$ photos, zebras $\to$ horses or summer $\to$ winter and vice versa).  



In order to incorporate additional prior knowledge to the architecture we add another loss term based on landmark predictions for both, the real faces and the cartoon faces.
To extend the usages of our model we additionally provide a one to ten mapping of real faces to cartoon faces. We make the whole architecutre conditional and train the models with ten different hair colors, so the user is free to choose his or her hair color.


### Materials and Methods
#### cycleGAN
To achieve the unpaired style transfer from real faces to cartoon faces, we have to learn the mapping function $G: A \to B$, with domain $A$ being real faces and domain $B$ being cartoon faces, such that the distribution of $G(A)$ is as similar as possible to the distribution $B$ by minimizing adversarial loss. Since this mapping is highly under-constrained, Jun-Yan Zhu et al. introduced the coupling of another mapping $H: B \to A$. The main idea is to minimize the consistency cycle loss $L(A, (H(G(A)))$ as well as the standart adversarial losses.  
![scheme](doc/images/cycleGAN_scheme_extended.png)  
The above figure is partly adapted from Jun-Yan Zhu et al. [2] and shows the method schematically. The reverse mapping with starting in $B$ is not shown.  

[The following would be the short visualization:]
![scheme_short](doc/images/cycle_GAN_scheme.png)
#### Datasets
[...] (Jan)

### First Results
[...] (Ben)

### Landmarks Loss
[...] (Ben)

### Conditions on Hair Color
[...] (Guodong)

### Improved cycleGAN
[...] (Gautam, with example!)

### Conclusion and Outlook
[...] (Gautam)

### Bibliography
[1]  
[2] Figure is avaiable on: https://github.com/eriklindernoren/PyTorch-GAN#cyclegan
[3]  
[...] (everyone who needs citations etc.)